# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mukalla,14.5425,49.1242,28.71,73,77,3.15,YE,1724902737
1,1,port lincoln,-34.7333,135.8667,21.39,50,0,9.29,AU,1724902738
2,2,cidade velha,14.9167,-23.6167,26.26,83,20,1.54,CV,1724902739
3,3,puerto natales,-51.7236,-72.4875,2.85,87,33,3.35,CL,1724902740
4,4,weno,7.4515,151.8468,28.97,84,100,2.06,FM,1724902742


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Humidity"]
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,mont-dore,-22.2833,166.5833,24.90,51,0,1.60,NC,1724902755
94,94,treasure island,27.7692,-82.7690,26.01,95,0,1.54,US,1724902768
143,143,bozova,37.3625,38.5267,24.38,68,0,1.86,TR,1724902906
146,146,messaad,34.1543,3.5031,25.90,33,0,3.25,DZ,1724902911
193,193,sena madureira,-9.0656,-68.6569,23.73,53,0,0.79,BR,1724902965


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,mont-dore,NC,-22.2833,166.5833,51,
94,treasure island,US,27.7692,-82.7690,95,
143,bozova,TR,37.3625,38.5267,68,
146,messaad,DZ,34.1543,3.5031,33,
193,sena madureira,BR,-9.0656,-68.6569,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "catagories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mont-dore - nearest hotel: No hotel found
treasure island - nearest hotel: No hotel found
bozova - nearest hotel: No hotel found
messaad - nearest hotel: No hotel found
sena madureira - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
divrigi - nearest hotel: No hotel found
berlin - nearest hotel: No hotel found
tambopata - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
vernal - nearest hotel: No hotel found
mahdia - nearest hotel: No hotel found
san ignacio - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
embi - nearest hotel: No hotel found
pucallpa - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
15,mont-dore,NC,-22.2833,166.5833,51,No hotel found
94,treasure island,US,27.7692,-82.7690,95,No hotel found
143,bozova,TR,37.3625,38.5267,68,No hotel found
146,messaad,DZ,34.1543,3.5031,33,No hotel found
193,sena madureira,BR,-9.0656,-68.6569,53,No hotel found
224,al jawf,SA,29.5000,38.7500,40,No hotel found
304,divrigi,TR,39.3710,38.1137,57,No hotel found
341,berlin,DE,52.5244,13.4105,88,No hotel found
355,tambopata,PE,-12.7333,-69.1833,57,No hotel found
380,newman,US,37.3138,-121.0208,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)